In [1]:
import sys
mins = 60

from hmmlearn import hmm

import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings
import sys
sys.path.append("../code/")
%matplotlib inline
warnings.filterwarnings("ignore")



In [2]:
from nilmtk import *
import os
import nilmtk
from nilmtk.disaggregate.fhmm_exact import sort_learnt_parameters
from nilmtk.disaggregate.fhmm_exact import create_combined_hmm
from nilmtk.disaggregate.fhmm_exact import FHMM

In [3]:
from dataloader import APPLIANCE_ORDER, get_train_test
num_folds =5
dataset = 1
train, test = get_train_test(dataset, num_folds=num_folds, fold_num=0)



In [4]:
def learn_hmm_appliance(train_appliance, num_states):
    """
    train_appliance: M homes, D days, 24 hours
    """
    mod = hmm.GaussianHMM(num_states, "full")
    t = train_appliance.copy()
    t = t.reshape(-1, 24, 1)
    t = t[np.random.choice(t.shape[0], 400, replace=False), :, :]
    mod.fit(t)
    return mod


In [10]:
def learn_fhmm(train, num_states):
    models = {}
    for appliance_num, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        models[appliance] = learn_hmm_appliance(train[:, 1:, :, :][:, appliance_num, :, :], num_states)
        print(appliance)
        print(models[appliance].means_)
        print(" "*20)
    
    new_learnt_models = OrderedDict()
    for appliance, appliance_model in models.items():
        startprob, means, covars, transmat = sort_learnt_parameters(
                        appliance_model.startprob_, appliance_model.means_,
                        appliance_model.covars_, appliance_model.transmat_)
        new_learnt_models[appliance] = hmm.GaussianHMM(
                    startprob.size, "full", startprob, transmat)
        new_learnt_models[appliance].means_ = means
        new_learnt_models[appliance].covars_ = covars

    learnt_model_combined = create_combined_hmm(new_learnt_models)

    f = FHMM()
    f.model = learnt_model_combined
    f.individual = new_learnt_models
    return f

In [13]:
def make_fhmm_preds(num_states):
    out = []
    for cur_fold in range(5):
        print(cur_fold)
        print("------------")
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        f = learn_fhmm(train, num_states)
        print("Training done for fold")
        pred = np.zeros_like(test[:, 1:, :, :])
        for home in range(pred.shape[0]):
            for day in range(pred.shape[2]):
                pred_home_day = f.disaggregate_chunk(pd.Series(test[home, 0, day, :]))[APPLIANCE_ORDER[1:-1]].values.T
                for appliance_num in range(pred.shape[1]):
                    pred[home, appliance_num, day, :] = pred_home_day[appliance_num, :].flatten()
    
        out.append(pred)
    return np.concatenate(out)



In [14]:
p = make_fhmm_preds(2)

0
------------
hvac
[[  329.73257278]
 [ 1603.15181081]]
                    
fridge
[[  70.57047786]
 [ 133.82260851]]
                    
dr
[[  3.13596462e+02]
 [  7.29118406e-02]]
                    
dw
[[   0.        ]
 [ 126.78631391]]
                    
mw
[[  2.33471045]
 [ 55.07983399]]
                    
Training done for fold
1
------------
hvac
[[  323.44190581]
 [ 1538.18995775]]
                    
fridge
[[ 123.91868279]
 [  66.225594  ]]
                    
dr
[[ 62.49172735]
 [ 62.49172735]]
                    
dw
[[   0.        ]
 [ 137.96406917]]
                    
mw
[[  2.44780625]
 [ 58.09052093]]
                    
Training done for fold
2
------------
hvac
[[  331.20918859]
 [ 1551.31511615]]
                    
fridge
[[  69.60910696]
 [ 123.28273777]]
                    
dr
[[  6.75118613e-01]
 [  6.92185654e+02]]
                    
dw
[[   0.        ]
 [ 126.83723123]]
                    
mw
[[  2.79074659]
 [ 61.27065196]]
                 

In [15]:
p.shape

(68, 5, 112, 24)

In [16]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err ={}
gt = tensor[:, 1:, :, :]


for num_states in range(2, 5):
    pred = make_fhmm_preds(num_states)
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err[num_states] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

0
------------
hvac
[[ 1740.88520735]
 [  377.05820963]]
                    
fridge
[[  67.23822595]
 [ 129.9931051 ]]
                    
dr
[[  5.60218166e-13]
 [  2.04225338e+02]]
                    
dw
[[ 16.37561457]
 [ 16.37561457]]
                    
mw
[[ 51.91049859]
 [  2.36549607]]
                    
Training done for fold
1
------------
hvac
[[  274.0966562 ]
 [ 1415.88052116]]
                    
fridge
[[  66.90923761]
 [ 126.98931603]]
                    
dr
[[  6.91551604e-01]
 [  7.72235957e+02]]
                    
dw
[[   0.        ]
 [ 115.02735658]]
                    
mw
[[  2.64767246]
 [ 55.24172608]]
                    
Training done for fold
2
------------
hvac
[[  306.92556987]
 [ 1522.67226513]]
                    
fridge
[[ 124.29879746]
 [  65.24816363]]
                    
dr
[[   0.        ]
 [ 180.86595189]]
                    
dw
[[ 15.53285764]
 [ 15.53285764]]
                    
mw
[[ 57.57467468]
 [  2.82111479]]
                   

In [17]:
pd.DataFrame(err)

,2,3,4
dr,208.195077,266.600572,268.861028
dw,41.388431,64.378195,131.184351
fridge,36.223373,42.233019,41.160048
hvac,353.128836,384.850738,420.464487
mw,22.628869,26.050713,41.136682
